<a href="https://colab.research.google.com/github/czcly0/BasketballPlayerStats/blob/master/CertiK_OA_Longyuan_Chu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 1. Finding the nonce. 

### Steps
    1, input a string
    2, randomly generate a nonce whose length is 100 - length of input, and concatenate the input and the nonce 
    3, compute the SHA265 hash of the result of step one and check if it starts with 4 ‘0’s
            3.1, if so, we get the result
            3.2, if not, repeat step 2 and step 3

In [ ]:
import random
import string
import hashlib

# verify if the hash given is satisfied of the given difficulty
def isValid(hashRes):
    return hashRes.startswith('0' * 4)

# randomly generate a string with the given length
def nonce_generator(size = 100, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

def findNonce(string):
    nonce = '0'
    hashRes = '0'
    while (not isValid(hashRes)):
        nonce = nonce_generator(100 - len(string))
        ipt = string + nonce
        hashRes = hashlib.sha256(ipt.encode()).hexdigest()
    return nonce, hashRes

string = 'a'
nonce, hashRes = findNonce(string)
print(nonce)
print(hashRes)

7P5PJVP9KVI3MFL246H9R93PE8IFA626QJWXS1O8D5EBTKB652DRDXYX1IBSKYRQFTYV6XMQSX1515QDFTUELX2FTA0WSY4LR6Z
0000483f31711101d4382da4a8ba478d5aee3f268e85a45242cb9485da8327b1


## Exercise 2. Constructing and verifying a blockchain

### Descriptions:
    1, Construct two classes, Block and Blockchain, for the future using
        1.1, Block
            1.1.1, Attributes: hashcode of previous block, the id of miner, the final nonce
            1.1.2, Method: compute_hash: a function to compute the hashcode of current string
        1.2, Blockchain
            1.2.1, Attributes: 
                1.2.1.1, chain → a list that contains all blocks computed so far
                1.2.1.2, difficulty → the number of zeros a valid hash starts with
            1.2.2, Methods:
                1.2.2.1, create_genesis_block: a function to generate the genesis block
                1.2.2.2, last_block: get the latest block of current blockchain
                1.2.2.3, proof_of_work: given a block, generate the satisfied hash
                1.2.2.4, add_block: verify and add the valid block to the current blockchain
                1.2.2.5, is_valid_proof: verify if the block is satisfied of the given difficulty
                1.2.2.6, mineTheNextBlock: given the last block and minerId, construct the next block
    2, verifyChain: given a blockchain, verify if all the blocks in the chain that satisfy the given difficulty
### Steps:
    1, Use the above two classes to initialize a block chain and repeatedly call mineTheNextBlock function to generate the next block and store it in the blockchain
    2, use verifyChain to verify if this blockchain is valid

In [ ]:
from hashlib import sha256

class Block:
    def __init__(self, previous_hash, miner=0, nonce='0'):
        self.miner = miner
        self.previous_hash = previous_hash
        self.nonce = nonce
    
    # a function to compute the hashcode of current string
    def compute_hash(self):
        block_string = self.previous_hash + str(self.miner) + str(self.nonce)
        return sha256(block_string.encode()).hexdigest()

class Blockchain: 
    difficulty = 4
    # constructor
    def __init__(self):
        self.chain = []
        self.create_genesis_block()
    
    # create the genesis_block
    def create_genesis_block(self):
        genesis_block = Block("")
        genesis_block.hash = self.proof_of_work(genesis_block)
        self.chain.append(genesis_block)
    
    # get the latest block of current blockchain
    @property
    def last_block(self):
        return self.chain[-1]
    
    # given a block, generate the satisfied hash
    def proof_of_work(self, block):
        computed_hash = block.compute_hash()
        while not computed_hash.startswith('0' * Blockchain.difficulty):
            block.nonce = nonce_generator(100 - len(block.previous_hash) - len(str(block.miner)))
            computed_hash = block.compute_hash()
        return computed_hash
    
    # verify and add the valid block to the current blockchain
    def add_block(self, block, proof):
        previous_hash = self.last_block.hash
        
        # this is not required, double check if the current block is constructed by the last block in this chain
        #if previous_hash != block.previous_hash:
            #return False
        
        # verify if the block if satisfied of the given difficulty
        if not self.is_valid_proof(block, proof):
            return False
        block.hash = proof
        self.chain.append(block)
        return True
    
    # verify if the block if satisfied of the given difficulty
    def is_valid_proof(self, block, block_hash):
        return (block_hash.startswith('0' * Blockchain.difficulty) and
                block_hash == block.compute_hash())
 
    def mineTheNextBlock(self, miner = 0):
        last_block = self.last_block
        
        # generate a new block based on the previous block
        new_block = Block(previous_hash=last_block.hash, miner=miner)
        
        # compute the hash of the new block
        proof = self.proof_of_work(new_block)
        self.add_block(new_block, proof)

# given a blockchain, verify if all the blocks in the chain that satisfy the given difficulty
def verify_chain(block_hash):
     return block_hash.startswith('0' * 4)

In [ ]:
# initialization
blockChain = Blockchain()

# generate 10 blocks in the chain
for _ in range(10):
    blockChain.mineTheNextBlock()

# a list to store the nonce and miner of each block in the blockchain 
res = []

# a list to store the result of verification 
verify = []

for block in blockChain.chain:
    verify.append(verify_chain(block.compute_hash()))
    res.append({'nonce':block.nonce, 'miner':block.miner})

In [ ]:
res

[{'nonce': '63HTV6ZU0WX1BDEHVCNB5E3AJYRJ8NH7F1DP38X6Y3DYF14P6LMES2NKBEA2UXMAOHH7NPP6YEK8EHHU5IEGCZ44ZPS7CYMUHQV',
  'miner': 0},
 {'nonce': '1G6RQO4SKW7SL8Q1LZYJCUQVN9FCAF48LNN', 'miner': 0},
 {'nonce': 'S3XGGLX78WZBLZS6D15A270ML20WH9MQNKY', 'miner': 0},
 {'nonce': '2CNO8TK0XYKHCN2YXBFQCUXCS6ORGX7S2L7', 'miner': 0},
 {'nonce': 'V2ESGPLP8G01Z1MW01QVB5Z982Y5HRNU61P', 'miner': 0},
 {'nonce': 'PVTM8MV4SRAJHEKOLY700SFAGTNRN9U7HV8', 'miner': 0},
 {'nonce': 'I6GJ5OX8X5K8AZ5P9L1T6JXR1K75EQPQ45B', 'miner': 0},
 {'nonce': 'AWGQSLEQYCJTHXIXUDKXBIYUB4E082TKT6Q', 'miner': 0},
 {'nonce': 'CGATZFFCIJU6NQ3129P1QL0B06SVF4SSBSG', 'miner': 0},
 {'nonce': '9TSE03D9YXUV9ACY1JOILJUFD7C2B04067X', 'miner': 0},
 {'nonce': 'ISS86L42JZR9QFTPW9Q4H59RLXV54PVSZJC', 'miner': 0}]

In [ ]:
verify

[True, True, True, True, True, True, True, True, True, True, True]

## Exercise 3. Multithreading

### Descriptions:
	1, input: the output format needed, the number of threads to use
	2, do_work: a function for each thread to output the information based on the given format
	3, main function:
		3.1, pattern: define the information format for each thread to output
		3.2, stop_threads: a monitor to check if we need to stop all threads, initialized to False. When it comes to be True, stop
		3.3, threads: a list containing all threads generated
		3.4, k: the number of threads we want to use
### Steps:
    1, initialize all the variables in main function
    2, generate k threads, pass do_work to the threads, and start the threads
    3, when needed, main thread can set stop_threads as True to stop all other thread


In [ ]:
import threading
from datetime import datetime
import time

# a function for each thread to output the information based on the given format
def do_work(pattern, id, stop):
    while True:
        print(pattern.format(datetime.now(), id))
        time.sleep(1)
        # if stop is True, stop the current thread
        if stop():
            print("Thread-{} Exiting loop.\n".format(id))
            break

def main():
    # define the information format for each thread to output
    pattern = 'Thread-{1}: [{1}] -- time: {0:%b} {0:%d} {0:%H}:{0:%M}:{0:%S} {0:%Y} \n'
    
    # a monitor to check if we need to stop all threads, initialized to False. When it comes to be True, stop
    stop_threads = False
    
    # a list containing all threads generated
    threads = []
    
    # the number of threads we want to use
    k = 3
    
    for id in range(k):
        #For something lightweight (no subclassing of Thread, no global variable), a lambda callback is an option
        tmp = threading.Thread(target=do_work, args=(pattern, id, lambda: stop_threads), name=id)
        threads.append(tmp)
        tmp.start()
    time.sleep(3)
    print('main: done sleeping; time to stop the threads.\n')
    stop_threads = True
    for thread in threads:
        thread.join()
    print('Finis.')
main()

Thread-0: [0] -- time: Jul 26 03:47:03 2021 

Thread-1: [1] -- time: Jul 26 03:47:03 2021 

Thread-2: [2] -- time: Jul 26 03:47:03 2021 

Thread-0: [0] -- time: Jul 26 03:47:04 2021 
Thread-2: [2] -- time: Jul 26 03:47:04 2021 

Thread-1: [1] -- time: Jul 26 03:47:04 2021 


Thread-2: [2] -- time: Jul 26 03:47:05 2021 
Thread-1: [1] -- time: Jul 26 03:47:05 2021 
Thread-0: [0] -- time: Jul 26 03:47:05 2021 



main: done sleeping; time to stop the threads.

Thread-2 Exiting loop.
Thread-1 Exiting loop.
Thread-0 Exiting loop.



Finis.


## Exercise 4. Inter-thread communication

## Descriptions:
	1, input: the number of threads to use
	2, send_receive: a function for each thread to send and receive the information
		For example, it is the i-th thread now
        2.1, print 
                Thread-i sending message to all other threads  
        2.2, insert the the threadId i into the queue
        2.3, read threadId from the queue if one is immediately available, otherwise, q will throw out an Empty Exception,         we continue reading, until we read k - 1 items from the queue
        2.4, once get an threadId j, print 
                Thread- i received message from Thread- j 
	3, main function:
		3.1, q: a global synchronized queue to store the information shared by threads
		3.2, threads: a list to store threads created
		3.3, k: the number of threads we want to use
## Steps:
    1, initialize all the variables in main function
    2, generate k threads, pass send_receive to each threads, and start all the threads
    3, make all threads join back the main thread and make sure they print all messages in q


In [ ]:
from queue import Queue
import threading
import time

# a function for each thread to send and receive the information
def send_receive(q, k, i):
    print("Thread-{0} sending message to all other threads  \n".format(i))
    for _ in range(k - 1):
        q.put(i)
    
    # wait 3 seconds to make sure every other threads start
    time.sleep(3)
    
    # using a set to check if receive this message before
    received = set()
    
    # the count of messages the current thread gets so far
    count = 0
    while True:
        try:
            # read threadId from the queue if one is immediately available, otherwise, continue reading
            cur_data = q.get(block=False)
            
            # if the current get threadId is same as this threadId, it means we get the info sent by itself,
            # we need to put it back
            # if the current read threadId is in the set, it means we already get the info from this thread,
            # we need to put it back
            if cur_data == i or cur_data in received:
                q.put(cur_data)
            else:
                # it is a valid info, we put it into the set and print it out
                received.add(cur_data)
                print("Thread-{0} received message from Thread-{1}  \n".format(i, cur_data))
                count += 1
                # if the count is k - 1, it means we already get all the info sent from others
                if count == k - 1:
                    break
        except:
            pass

def main():     
    # Create the shared queue and launch both threads
    q = Queue()
    threads = []
    k = 3
    for i in range(k):
        tmp = threading.Thread(target = send_receive,
                               args = (q, k, i))
        threads.append(tmp)
    for i in range(k):
        threads[i].start()
    for i in range(k):
        threads[i].join()
main()

Thread-0 sending message to all other threads  

Thread-1 sending message to all other threads  

Thread-2 sending message to all other threads  

Thread-0 received message from Thread-1  
Thread-1 received message from Thread-0  

Thread-1 received message from Thread-2  
Thread-2 received message from Thread-1  


Thread-2 received message from Thread-0  


Thread-0 received message from Thread-2  



## Exercise 5. Decentralizing the blockchain

### Descriptions:
    1, Construct two classes for the future using
        1.1, Block
            1.1.1, Attributes:
                1.1.1.1, Id of miner
                1.1.1.2, hashcode of previous block, default value is  “”
                1.1.1.3, nonce, default value is  “0”
                1.1.1.4, hash, final hash in the this block
            1.1.2, Method: compute_hash: a function to compute the hashcode of current string
        1.2, Blockchain
            1.2.1, Attributes: 
                1.2.1.1, chain → a list that contains all blocks computed so far
                1.2.1.2, difficulty → the number of zeros a valid hash starts with
                1.2.1.3, flag → a boolean value to denote if we need to create a genesis block
            1.2.2, Methods:
                1.2.2.1, create_genesis_block: a function to generate the genesis block
                1.2.2.2, last_block: get the latest block of current blockchain
                1.2.2.3, proof_of_work: given a block, generate the satisfied hash
                1.2.2.4, mineTheNextBlock: given the last block and minerId, compute and validate the proof for the next block
                1.2.2.5, optional, we can set a timer inside this function to control the time used to find the next mine
                    I, if the new proof is valid, add it to the current blockchain and return True
                    II, otherwise, return False
    2, verifyChain: given a blockchain, verify if all the blocks in the chain that satisfy the given difficulty
    3, is_valid_proof: verify if the hash given is satisfied of the given difficulty
    4, copyBlockchain: a function to deep copy the given blockchain
    5, nonce_generator: randomly generate a string with the given length
    6, start_work: a function for each thread to create a blockchain, mine the new blocks and communicate with other threads 
        For example, it is the i - th thread now
        6.1, initialize a blockchain
        6.2, while the length of this chain is smaller than 10 blocks
            6.2.1,  use mineTheNextBlock to get the new block
            6.2.2, if the return value of mineTheNextBlock is True, deep copy the current blockchain and put it into the queue
            6.2.3, reading all chains in the queue and put all valid chains into a list, incomingChains
            a, if the miner of the last block in the incoming chain is not i, put this chain into incomingChains
            b, else, break the loop
            6.2.3, if we get any incoming chain
                 and if every block in the incoming chain is validate 
                 and if the length of incoming chain is longer than the current chain 
             → replace the current chain by the incoming chain
        6.3, if the current chain just have 10 block, put it into the result, and set the threadControl event to stop all other threads
    7, main function:
        7.1, q: a global synchronized queue to store the information shared by threads
            7.2, threads: a list to store threads created
            7.3, k: the number of threads we want to use
            7.4, targetNumber: the number of blocks we want to mine
            7.5, threadControl: an Event object to monitor and control if we need to stop all threads other than the main thread
            7.6, result: a global synchronized queue to store the final blockchain from these threads
            7.7, queueLock: a Lock object to make sure there will not any interruption while a thread is reading incoming chains from q


### Steps:
    1, initialize all the variables in the main function and activate all threads to run start_work
    2, once the threadControl is set, make stop_threads to True, and stop all other threads
    3, get the final blockchain from the result. Verify and print all blocks in the chain


In [ ]:
from queue import Queue
import threading
import time
from datetime import datetime

from hashlib import sha256

import random
import string

class Block:
    def __init__(self, miner, previous_hash="", nonce='0'):
        self.miner = miner # Id of miner
        self.previous_hash = previous_hash # hashcode of previous block
        self.nonce = nonce
        block_string = previous_hash + str(miner) + str(nonce)
        self.hash = sha256(block_string.encode()).hexdigest()

    #  a function to compute the hashcode of current string
    def compute_hash(self):
        block_string = self.previous_hash + str(self.miner) + str(self.nonce)
        return sha256(block_string.encode()).hexdigest()

class Blockchain: 
    difficulty = 4
    def __init__(self, miner, flag = True):
        self.miner = miner # Id of miner
        self.chain = [] # a list that contains all blocks computed so far
        # if flag is True, it means we are creating a blockchain from scratch and we need to create a genesis block
        # if flag is False, it means we are creating a copy of other blockchain 
        # and we do not need to create the genesis block
        if flag:
            self.create_genesis_block(miner)
    # a function to generate the genesis block
    def create_genesis_block(self, miner):
        genesis_block = Block(miner)
        genesis_block.hash = self.proof_of_work(genesis_block, miner)
        self.chain.append(genesis_block)
    
    @property
    def last_block(self):
        return self.chain[-1]
    
    # given a block, find the valid hash
    def proof_of_work(self, block, miner):
        computed_hash = block.compute_hash()
        #t1 = time.time()
        while not computed_hash.startswith('0' * Blockchain.difficulty):
            # optional, using to set a timer to contral the time used to get a mine
            '''t2 = time.time()
            if t2 - t1 > 3:
                return None'''
            block.nonce = nonce_generator(100 - len(block.previous_hash) - len(str(block.miner)))
            computed_hash = block.compute_hash()
        return computed_hash
    
    # given minerId, compute and validate the proof for the next block
    def mineTheNextBlock(self, miner):                              
        last_block = self.last_block
        new_block = Block(previous_hash=last_block.hash, nonce=nonce_generator(), miner=miner)
        proof = self.proof_of_work(new_block, miner)
        # if the new block is valid, add it to the chain
        if is_valid_proof(new_block, proof):
            new_block.hash = proof
            self.chain.append(new_block)
            return True
        else:
            return False

# given a blockchain, verify if all the blocks in the chain that satisfy the given difficulty     
def verifyChain(blockChain):
    for block in blockChain.chain:
        if not is_valid_proof(block, block.hash):
            return False
    return True

# verify if the hash given is satisfied of the given difficulty
def is_valid_proof(block, block_hash):
    if not block_hash:
        return False
    return (block_hash.startswith('0' * 4) and
                block_hash == block.compute_hash())

# deep copy the given blockchain
def copyBlockchain(blockchain):
    newChain = []
    for block in blockchain.chain:
        tmp = Block(miner=block.miner, previous_hash=block.previous_hash[:], nonce=block.nonce[:])
        newChain.append(tmp)
    newBc = Blockchain(blockchain.miner, False)
    newBc.chain = newChain
    return newBc

def nonce_generator(size = 100, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

def start_work(id, stop, threadControl, targetNumber):
    print("Thread-{0} get to work  \n".format(id))
    # initialize a blockchain
    blockChain = Blockchain(id)
    
    while len(blockChain.chain) < targetNumber:        
        # if True, it means we already get the target chain, stop the current process
        if stop():
            return
        
        # if the return value of mineTheNextBlock is True, deep copy the current blockchain and put it into q
        if blockChain.mineTheNextBlock(id):
            q.put(copyBlockchain(blockChain))
        
        # a list to store all incoming chain
        incomingChains = []
        
        # lock to make sure there will not any interruption while a thread is reading incoming chains from q
        queueLock.acquire()
        while True:
            try:
                 # read threadId from the queue if one is immediately available, otherwise, continue reading
                incomingChain = q.get(block=False)
                
                # if the miner of the last block in the incoming chain is i,
                # it means the incoming chain is the current chain, we need to put it back and stop reading
                if incomingChain.miner == blockChain.miner:
                        q.put(incomingChain)
                        break
                incomingChains.append(incomingChain)

            except:
                break
        queueLock.release()
        if len(incomingChains) > 0:
            for incomingChain in incomingChains:
                # if every block in the incoming chain is valid and the length of incoming chain is longer than the current chain
                # replace the current chain with the incoming chain
                if verifyChain(incomingChain) and len(incomingChain.chain) > len(blockChain.chain) and len(incomingChains) <= targetNumber: 
                    blockChain = incomingChain
    
    # if the current chain has the corrct number of blocks, add this chain to result
    if len(blockChain.chain) == targetNumber:
        result.put(blockChain)
        threadControl.set()

In [ ]:
# a global synchronized queue to store the information shared by threads
q = Queue()

threads = [] # a list to store threads created
k = 5 # number of threads
targetNumber = 10 # number of blocks need to build

# an Event object to monitor and control if we need to stop all threads other than the main thread
threadControl = threading.Event()

# a global synchronized queue to store the final blockchain from these threads
result = Queue()

# a monitor to check if we need to stop all threads, initialized to be False. 
# When it comes to be True, will stop all threads except the main thread
stop_threads = False

# a Lock object to make sure there will not any interruption while a thread is reading incoming chains from q
queueLock = threading.Lock()

# initialize and start all threads
for i in range(k):
    tmp = threading.Thread(target = start_work, 
                           args=(i,
                                 lambda: stop_threads,
                                 threadControl,
                                 targetNumber), 
                           name = i
                          )
    threads.append(tmp)
    tmp.start()

while True:
    # once the Event threadControl is set, stop all other threads
    if threadControl.isSet():
        print('Already got the chain, stop all the threads')
        stop_threads = True
        break
        
for thread in threads:
    thread.join()
    
print('Finis.')

Thread-0 get to work  

Thread-1 get to work  

Thread-2 get to work  

Thread-3 get to work  

Thread-4 get to work  

Already got the chain, stop all the threads
Finis.


In [ ]:
blockChain = result.get()
res = []
proof = []
for block in blockChain.chain:
    res.append({'nonce':block.nonce, 'miner':block.miner})
    proof.append(block.hash)

In [ ]:
res

[{'nonce': '2NIEZEAD1ZLRQP6SSYG3NFN4OI9L8QSBVPV8R9GIVHQEKNQT8LI92R92PJJPGRYVTD56KZL4PEU95C9EKAH40KKPOVVQ7MV7V3N',
  'miner': 0},
 {'nonce': 'QOMOYOOPIRLJS3HDAH1B41WWEL0ATGRNERI', 'miner': 0},
 {'nonce': 'PLO4SYCSXBUM3EDP22RAZYJMXCLNYHE126U', 'miner': 0},
 {'nonce': 'TSP54RRA7M775UC8L8ZFX07PBZ56N8U8GC9', 'miner': 0},
 {'nonce': 'U7X0KHWEDMVFOCSZBVAHR26MR2XF08U51YA', 'miner': 3},
 {'nonce': 'W275WY7M1RUBFWBB53LSU7QR6LIDCVXLR35', 'miner': 1},
 {'nonce': 'NMP5MYSJW10OHOJB4XD4IP1AIPL32JY9VO2', 'miner': 1},
 {'nonce': 'QKSCV4O2FQT8GUUEOHZD11LY3IFUXKMQ21C', 'miner': 1},
 {'nonce': 'MSA0SEL4T1GRQTJIYLCYJ1AEQOXX0T8M17V', 'miner': 2},
 {'nonce': 'QH0J2XDJ1996AN3N1CL2B9OZQLDG6LC3SWK', 'miner': 2}]

In [ ]:
proof

['0000086d9290a6621857a594ad553862954ddcfcdaf89dcf5cf0088e6d4163d5',
 '00007f9786d588f41ad1640e97acdece24a5fbf9eb0288e3b332cbed0b5f396d',
 '000063c9277ee4c5de3c567a08905cfb433061ea210d56783046e6f6d089a346',
 '00008ce179ec3d5c555baba1cf47179a1586e5368c3a00dcf085a4162ddab800',
 '0000642b727b692ff5106eabada4a04ce83822fb83e2ab2b488e57afd65a9138',
 '0000a4fb69cd2204b28d91e3a83f36d62b4a61371daf48b2b8cbd5e036df8dfd',
 '00003d86f95108bece6b2a52b9762d785f79653159dfad92775515a0e46428ee',
 '00003e6c6da6da096ef82145f05214923ded9147dcbd90bd738035a164f6ae47',
 '0000d6fac023dd48c6d029f1029c512ff97e5b8b3e4528724bdd2602e9e4d94c',
 '000032a8acef8e74d877e1448a31fb32843bdda21f9d2d278303a215bb445f66']

### Does this blockchain scale? What happens when the number of threads go up?

Yes, I built two classes, Block and Blockchain, to make this blockchain eaier to scale

with the increasing number of threads used, in the beginning, the program will be faster, but it will slow down if we use too many threads

There are several limitation that may affect the performance of a program. For example, CPU resources and RAM resources are finite and any one can limit the performance of a program.

### If one thread stopped working, does it affect the blockchain? What if the thread comes back on later?

It depends. 

If this thread is a main thread, it depends on if the rest of other threads is deamon thread or not. All non-daemon threads will stop working if main thread stopped. And if there are not any alive non-daemon thread, the JVM will exit. 

If this thread is not a main thread, it generally will not affect others. But if all threads are in a same process, they will share heap, static memory segment and os resource. So it is still possible to affect the blockchain.

### What if we require the SHA256 hashes to start with more zeros?

It will take longer time to get the result

### Is it easy for a “malicious thread” to modify the miner fields in the chain

It depends on if this malicious thread is running on the same process with the chain. If so, since they are sharing same memory space, it is possible to modify the miner fields. But if they are running on different processes, it should be a lot more difficult.

### How long did it take?
Around 10 - 12 hours

### What’s the most difficult part? 
Since I do not have any on-field experience related to blockchain, so it takes me some time to understand the logic, concepts and terminoloies in blockchain area

### Any other feedback for the problem or the process?
They are interesting questions. 

And I can tell that this person who designs this OA must spend some time on designing those questions. The questions are ranked from easy to hard. And it feels like the first 4 questions are separated from the last question. So it is helpful for someone who does not familiar with the blockchain area to get started with.

After I complete this OA, I feel that I learned a lot about blockchain and muti-threading.